In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<ii> = NumberField(xx^2 + 1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [4]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 9.02 s, sys: 289 ms, total: 9.31 s
Wall time: 21.3 s


576

In [5]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 4.61 s, sys: 205 ms, total: 4.81 s
Wall time: 27.7 s


2

In [6]:
with open('all_L_sets.pickle', 'rb') as fil:
    all_L_sets = pickle.load(fil)

In [7]:
L_set_base = ('E1', 'G4', 'E2', 'G3', 'E3')
possible_L_sets = [L_set for L_set in all_L_sets if L_set[0:3] == ('E1', 'G4', 'E3')]

In [8]:
%time L_set_ideals, all_ideals = SE1.find_conditions_on_L_sets(possible_L_sets, "eck")

CPU times: user 42 s, sys: 52.2 ms, total: 42.1 s
Wall time: 42.1 s


In [9]:
L_set_ideals

[[('E1', 'G4', 'E3', 'G6', 'E6'), []],
 [('E1', 'G4', 'E3', 'G6', 'F45'), []],
 [('E1', 'G4', 'E3', 'G6', 'F24'),
  [Ideal (4*c + (-ii)*d + (ii)*e + (-ii + 2)*f, 5*d^2 + (-4*ii - 2)*d*e + (4*ii - 3)*e^2 + (8*ii - 6)*d*f + (36*ii + 38)*e*f + (-24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1,
   Ideal (4*c + (ii)*d + (-ii)*e + (ii + 2)*f, 5*d^2 + (4*ii - 2)*d*e + (-4*ii - 3)*e^2 + (-8*ii - 6)*d*f + (-36*ii + 38)*e*f + (24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1]],
 [('E1', 'G4', 'E3', 'F13', 'E6'), []],
 [('E1', 'G4', 'E3', 'F13', 'E5'), []],
 [('E1', 'G4', 'E3', 'F13', 'E2'), []],
 [('E1', 'G4', 'E3', 'G5', 'F46'), []],
 [('E1', 'G4', 'E3', 'G5', 'E5'), []],
 [('E1', 'G4', 'E3', 'G5', 'F24'),
  [Ideal (4*c + (ii)*d + (ii + 2)*e + (-ii)*f, 5*d^2 + (-8*ii - 6)*d*e + (24*ii - 7)*e^2 + (4*ii - 2)*d*f + 

In [10]:
ideales = [_ for i in range(len(all_ideals))]
e_sosts = [_ for i in range(len(all_ideals))]
d_sosts = [_ for i in range(len(all_ideals))]
for i in range(len(all_ideals)):
    ideales[i] = all_ideals[i]
    poly_e = ideales[i].gens()[0]
    e_sosts[i] = {e: -(poly_e-poly_e.coefficient(e)*e)/poly_e.coefficient(e)}
    poly_d = [gen for gen in ideales[i].subs(e_sosts[i]).gens() if gen !=0][0]
    d_sosts[i] = {d:-(poly_d-poly_d.coefficient(d)*d)/poly_d.coefficient(d)}
    #print([gen.subs(e_sosts[i]).subs(d_sosts[i]) for gen in ideales[i].gens()])

In [11]:
cubics = [SE1.subs(e_sosts[i]).subs(d_sosts[i]) for i in range(len(ideales))]

In [90]:
def find_projective_equivalence(cubic1, cubic2, L_sets, param=None):
    if param is None:
        return find_projective_equivalence_without_parameters(cubic1, cubic2, L_sets)
    elif len(param) == 2:
        return find_projective_equivalence_with_two_parameters(cubic1, cubic2, L_sets, param)
    else:
        raise ValueError("Not yet supported")

def find_projective_equivalence_without_parameters(cubic1, cubic2, L_sets):
    for L_set in L_sets:
        proj = cubic2.find_projectivity(cubic2.L_set_base, L_set)[0]
        eqn = cubic2.eqn.subs(change_coordinates(proj))
        if cubic1.are_cubics_same(eqn):
            return L_set

def find_projective_equivalence_with_two_parameters(cubic1, cubic2, L_sets, param):
    vrs = cubic1.P.gens()[0:4]
    mon = (sum(vrs) ^ 3).monomials()
    l, m = P.gens()[-2:]
    param_sost = {param[0]:l, param[1]:m}
    param_cubic2 = cubic2.subs(param_sost)
    for L_set in L_sets:
        proj = param_cubic2.find_projectivity(cubic1.L_set_base, L_set)[0]
        eqn = param_cubic2.eqn.subs(change_coordinates(proj))
        eqn = [factor[0] for factor in eqn.factor() if any(var in vrs for var in factor[0].variables())][0]
        current_conds = list(set(matrix([[cubic1.coefficient(mn) for mn in mon], [eqn.coefficient(mn) for mn in mon]]).minors(2)))
        prim_dec = P.ideal(current_conds).radical().primary_decomposition()
        if [(ideal, L_set) for ideal in prim_dec if len(ideal.gens())==1] != []:
            L_set_ideal = (prim_dec[0], L_set)
            break
    poly = L_set_ideal[0].gens()[0]
    sost = {l: poly.coefficient(m), m: -poly.coefficient(l)}
    #print(cubic1.are_cubics_same(param_cubic2.eqn.subs(change_coordinates(proj)).subs(sost)))
    return L_set_ideal[1], {param[i]:list(sost.values())[i] for i in range(len(param))}

In [93]:
%%time
L_sets = [SE1.L_set_base]
for L_set in [get_permuted_L_set(perm) for perm in SE1.find_admissible_permutations()]:
    if L_set not in L_sets:
        L_sets.append(L_set)
for cubic in cubics[1:]:
    L_set, sost = find_projective_equivalence(cubics[0], cubic, L_sets, [c,f])
    print(L_set, sost)
    proj = cubic.find_projectivity(cubic.L_set_base, L_set)[0]
    print(cubics[0].are_cubics_same(cubic.eqn.subs(change_coordinates(proj)).subs(sost)))

('E1', 'G4', 'E2', 'G6', 'F34') {c: (-ii)*c - f, f: -c + (-ii)*f}
True
('E1', 'G4', 'E2', 'G6', 'F34') {c: (2*ii + 1)*c + f, f: -c + (2*ii - 1)*f}
True
('E1', 'G4', 'E2', 'G3', 'E3') {c: (2*ii + 1)*c + (-2*ii - 1)*f, f: -5*c + (4*ii - 3)*f}
True
CPU times: user 6.8 s, sys: 43.9 ms, total: 6.84 s
Wall time: 6.91 s


In [12]:
SE1_ = [SE1.subs(e_sosts[0]).subs(d_sosts[0])]+[SE1.subs(e_sosts[i]).subs(d_sosts[i]).subs({c:l, f:m}) for i in range(1, len(all_ideals))]

In [71]:
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
current_conds = list(set(matrix([[SE1_[0].coefficient(mn) for mn in mon], [SE1_[3].coefficient(mn) for mn in mon]]).minors(2)))
prim_dec = P.ideal(current_conds).radical().primary_decomposition()
prim_dec

[Ideal (5*c*l + (-4*ii + 3)*f*l + (2*ii + 1)*c*m + (-2*ii - 1)*f*m) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1]

In [72]:
poly = prim_dec[0].gens()[0]
poly.coefficient(l), poly.coefficient(m)

(5*c + (-4*ii + 3)*f, (2*ii + 1)*c + (-2*ii - 1)*f)

In [73]:
sost1 = {l: poly.coefficient(m), m: -poly.coefficient(l)}
print(sost1)
SE1_[0].are_cubics_same(SE1_[3].subs(sost1))

{l: (2*ii + 1)*c + (-2*ii - 1)*f, m: -5*c + (4*ii - 3)*f}


True

In [53]:
%%time 
L_sets = [SE1.L_set_base]+list(set([get_permuted_L_set(perm) for perm in SE1_[1].find_admissible_permutations()]))
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
for L_set in L_sets:
    proj = SE1_[2].find_projectivity(SE1_[0].L_set_base, L_set)[0]
    eqn = SE1_[2].eqn.subs(change_coordinates(proj))
    eqn2= list(eqn.factor())[-1][0]
    eqn = [factor[0] for factor in eqn.factor() if any(var in vrs for var in factor[0].variables())][0]
    print(eqn==eqn2)
    current_conds = list(set(matrix([[SE1_[0].coefficient(mn) for mn in mon], [eqn.coefficient(mn) for mn in mon]]).minors(2)))
    prim_dec = P.ideal(current_conds).radical().primary_decomposition()
    if [(ideal, L_set) for ideal in prim_dec if len(ideal.gens())==1] != []:
        L_set_ideal = (prim_dec[0], L_set)
        break
L_set_ideal

True
True
CPU times: user 1.19 s, sys: 8.03 ms, total: 1.2 s
Wall time: 1.23 s


(Ideal (13*c*l + (-8*ii + 1)*f*l + (2*ii + 3)*c*m + (2*ii + 3)*f*m) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1,
 ('F14', 'E1', 'G5', 'F56', 'G3'))

In [45]:
poly = L_set_ideal[0].gens()[0]
poly

13*l^2 + (6*ii + 4)*l*m + (-2*ii + 3)*m^2

In [47]:
sost = {l: poly.coefficient(m), m: -poly.coefficient(l)}
SE1_[0].are_cubics_same(SE1_[3].eqn.subs(change_coordinates(proj)).subs(sost))

False

In [20]:
SE1_1 = SE1_[0]

In [21]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 2.97 s, sys: 197 ms, total: 3.17 s
Wall time: 23.6 s


576

In [22]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 1.59 s, sys: 181 ms, total: 1.77 s
Wall time: 7.58 s


4

In [23]:
lines_perms = [Permutation(SE1_1.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE1_1]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(4, 'C4')

In [24]:
lines_perms_group.gens_small()

[(2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20)]

In [25]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46')]

In [26]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens() if perm.order()==4]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46'),
 ('E1', 'G4', 'F34', 'F16', 'E2', 'E6')]

In [27]:
lines_perms_group.gens()

[(),
 (2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20),
 (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (2,22,19,3)(4,23,7,21)(5,6,25,26)(8,11,13,16)(9,17,14,12)(18,20,27,24)]